In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import surprise

In [10]:
items = pd.read_csv('BX-Books.csv',sep=';',error_bad_lines=False, encoding = 'latin-1')
ratings = pd.read_csv('BX-Book-Ratings.csv',sep=';', error_bad_lines = False, encoding = 'latin-1')
ratings = ratings[ratings['Book-Rating'] > 0]
users_dataset = pd.read_csv('BX-Users.csv',sep=';', error_bad_lines = False, encoding = 'latin-1')

Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 fields, saw 9
Skipping line 251296: expected 8 fields, saw 9
Skipping line 259941: expected 8 fields, saw 9
Skipping line 261529: expected 8 fields, saw 9



In [11]:
ratings_final = ratings[ratings['ISBN'].isin(items['ISBN'])]

d = (len(ratings) / (len(ratings['User-ID'].unique())) * len(ratings['ISBN'].unique()))
print (d) 
print (len(ratings['User-ID'].unique()) , len(ratings['ISBN'].unique()) )

929865
(77805, 185973)


In [12]:
len(ratings['User-ID'].unique())

77805

In [13]:
len(ratings['ISBN'].unique())

185973

In [14]:
import sys
sys.version_info[0]

2

In [15]:
ratings_final = ratings.groupby('ISBN').filter(lambda x: len(x) >= 10)
d2 = (len(ratings_final)) / ((len(ratings_final['User-ID'].unique())) * len(ratings_final['ISBN'].unique()))
print d2
print len(ratings_final['User-ID'].unique()), len(ratings_final['ISBN'].unique())

0
40493 5645


In [16]:
b = ratings_final.groupby('User-ID').filter(lambda x: len(x) >= 20)
d3 = len(b) / (len(b['User-ID'].unique()) * len(b['ISBN'].unique()) )
print d3
print len(b['User-ID'].unique()), len(b['ISBN'].unique())

0
1152 5546


In [17]:
grouped = b.groupby('User-ID')
test = grouped.apply(lambda x: x.sample(10))
test = test.reset_index(drop=True)

In [18]:
keys = ['User-ID', 'ISBN']
i1 = b.set_index(keys).index
i2 = test.set_index(keys).index
train = b[~i1.isin(i2)]

In [19]:
train.to_csv('train.csv')
test.to_csv('test.csv')
b.to_csv('sampled.csv')

In [26]:
all_books = ratings_final['ISBN'].unique()

In [38]:
book_data = {}
import requests
k = 0

In [39]:
for isbn in all_books:
    data = requests.get("https://www.googleapis.com/books/v1/volumes?q=isbn:" + str(isbn))
    k += 1 
#     print data.json()
    book_data[isbn] = data.json()
    if k % 500 == 0 and k >= 500:
        print k

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500


In [41]:
import pickle
with open("full_book_data.pickle", "wb") as file:
    pickle.dump(book_data, file)

In [43]:
import random
random.choice(list(book_data.keys()))

u'0805029648'

In [44]:
book_data['0805029648']

{u'error': {u'code': 403,
  u'errors': [{u'domain': u'usageLimits',
    u'extendedHelp': u'https://code.google.com/apis/console',
    u'message': u'User Rate Limit Exceeded. Please sign up',
    u'reason': u'userRateLimitExceededUnreg'}],
  u'message': u'User Rate Limit Exceeded. Please sign up'}}

In [47]:
book_data[book_data.keys()[1]]

{u'error': {u'code': 403,
  u'errors': [{u'domain': u'usageLimits',
    u'extendedHelp': u'https://code.google.com/apis/console',
    u'message': u'User Rate Limit Exceeded. Please sign up',
    u'reason': u'userRateLimitExceededUnreg'}],
  u'message': u'User Rate Limit Exceeded. Please sign up'}}